# Google Art & Culture - Case study using CRISPS-DM

#### Autors: Manuel Alejandro Aponte, Cristian Beltran, Maria Paula Peña

In this notebook it will webscraping of the page Google Art & Culture

## Objectives
The objective of this notebooks is:

* Download images using webscraping.
* Download images metadata.
* Store all information in a datasheet.

## Prerequisites

* Familiarity with python 
* Lastest version of Google WebDriver, Source: https://chromedriver.chromium.org/
* Install python packages.
* Use VPN (Recomended)

## Background 
This notebook belongs to Google Art & Culture Case Study using CRIPS-DM, where would be include process such as webscraping, exploratory data analysis, ML classificators and dashboards. 

In [ ]:
#Import project packages
import sys
sys.path.append('../src/')
from Scraper import Scraper
from Parallel import parallel
#Import packages
from typing import List
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from pprint import pprint
from concurrent.futures import ThreadPoolExecutor
from time import sleep
import requests

# Global Settings

In [ ]:
#Colors of sections to be webscraped
#COLORS = ["WHITE","PINK","YELLOW","PURPLE","BLUE","TEAL","GREEN","ORANGE","RED","BROWN","BLACK"] 
COLORS = ["WHITE","PINK","YELLOW"] 
DRIVER_PATH = r"../chromedriver.exe"
DATA_RAW_FOLDER = "../data/raw"
DATA_PROCESSED_FOLDER = "../data/processed"
DATA_FINAL_FOLDER = "../data/final"


# Webscraping Attributes Extraction (Phase 1)

In [ ]:


def webscraping(color:str):
    """Perform webscraping of Google Art & Culture web page

    Parameters
    ----------
    color : str
        the color of target page
    """
    scraper = Scraper(DRIVER_PATH,color)
    scraper.open()
    data = scraper.scraping_data()
    return data
    
    
webscraping_data = parallel(webscraping,COLORS)

inspect some data

In [ ]:
#Print WHITE collection
print(webscraping_data[0][0:2])
#Print PINK collection
print(webscraping_data[1][0:2])

Merge Collections into one

In [ ]:
flatten_data = [item for sublist in webscraping_data for item in sublist]

In [10]:
flatten_data[0:2]

[{'url': 'https://lh3.googleusercontent.com/ci/AC_FhM_oasJFGZVi8jye_ImxogO_y6DHA6Ha2nK85qkgEdZiJd5ku_wyJ6AOyfEOKIW-rEsmFZB5ng=w313-c-h313-fcrop64=1,00000d37ffff8ce4-rw-v1',
  'data': '{"Title":" Symphony in White, No. 1 The White Girl","Creator":" James McNeill Whistler","Date Created":" 1862","External Link":"  For more information about this and thousands of other works of art in the NGA collection, please visit\xa0http//www.nga.gov/","Medium":" oil on canvas","Object Credit":" Harris Whittemore Collection","Dimensions":" overall 213 x 107.9 cm (83 7/8 x 42 1/2 in.)\\u000b\\u000bframed 244.2 x 136.5 x 8.3 cm (96 1/8 x 53 3/4 x 3 1/4 in.)","Classification":" Painting","Artist School":" American","Artist Nationality":" American","Artist Details":" American, 1834 - 1903"}',
  'category': 'WHITE'},
 {'url': 'https://lh3.googleusercontent.com/ci/AC_FhM9f2_H_Vib6NbUnODNBmKAF4_nxQENdrmuCgD_qbON7A8MhrGE8artJsUQA-1s0VCBUPmhAkQ=w313-c-h313-fcrop64=1,00000b77ffffdd97-rw-v1',
  'data': '{"Title"

In [11]:
df = pd.DataFrame(flatten_data)
df['index'] = df.index
df.to_csv(f'{DATA_RAW_FOLDER}/pictures_original.csv', index = False)  

# Webscraping Image Extraction (Phase 2)

Get the image_url and the identify key (index)

In [ ]:
   
df = df.read_csv(f'{DATA_RAW_FOLDER}/pictures_original.csv')
df = df[['index','url']]   
data = list(df.itertuples(index=False, name=None)) # Convert data into tuples

Apply parallel transformation for download files 

In [ ]:
def get_image(data):
    id = data[0]
    link = data[1]
    path = f'../data/raw/img/{id}.jpg'
    img = requests.get(link).content
    with open(path, 'wb') as handler:
        handler.write(img)
    return (id,f'{id}.jpg')

data_processed = parallel(get_image,data) 
print(data_processed[0:5])

Convert data into df and export it

In [ ]:
df_processed = pd.DataFrame(data_processed, columns=['id','filename']) #Parse data into df
df.to_csv(f'{DATA_PROCESSED_FOLDER}/pictures_images.csv', index = False)  # Export

# Transform

* Filtering, cleansing, de-duplicating, validating, and authenticating the data.
* Performing calculations, translations, or summarizations based on the raw data. This can  include changing row and column headers for consistency, converting currencies or other units of measurement, editing text strings, and more.
* Ensure data quality
* Formatting the data into tables or joined tables to match the schema of the target data warehouse.

In [ ]:
df_original = df.read_csv(f'{DATA_RAW_FOLDER}/pictures_original.csv')
df_images = df.read_csv(f'{DATA_PROCESSED_FOLDER}/pictures_images.csv')
df = df_original.merge(df_images, left_on='index', right_on='index')
print(df.head())

In [ ]:
print(df.shape)
print(df_original.shape)
print(df_images.shape)

In [ ]:
df.info()

In [ ]:
df.duplicated().value_counts() 

In [ ]:
df.isnull().sum()

In [ ]:
#TODO: MERGE DF INTO ONE 
#     PARSE DATA COLUMN 

In [ ]:
from tqdm.notebook import tqdm
import time
for i in tqdm(range(10)):
    time.sleep(1)